In [28]:
import warnings
warnings.filterwarnings('ignore')

# Handle table-like data and matrices
import numpy as np
import pandas as pd
import datetime as dt

In [29]:
confirmed_df = pd.read_csv("datasets/time_series_covid19_confirmed_global_modified.csv",sep=',')
tests_positive_df = pd.read_csv("datasets/covid-19-tests-vs-cases-positivity-comparisons.csv")
print("Tests positive shaped: ", tests_positive_df.shape)
print("Confirmed shape: ", confirmed_df.shape)

Tests positive shaped:  (14537, 5)
Confirmed shape:  (182, 96)


In [30]:
#Coveeting the string date into datetime pandas object
tests_positive_df['Date'] =  pd.to_datetime(tests_positive_df['Date'])

In [31]:
#Converting the date in dataframe to epoch. It would be easy to compare epoch times to make the data uniform 
#based on the starting and ending date
tests_positive_df['Date'] = (tests_positive_df['Date'] - dt.datetime(1970,1,1)).dt.total_seconds()

In [32]:
#Converting into a timeseries dataframe
jp_tests_positive_df = tests_positive_df.pivot_table(values="Total confirmed cases of COVID-19 (cases)",index="Entity",columns="Date")

In [33]:
#Finding the columns with the same dates in both the dataframes
similar_columns_list_confirmed = []
for col in jp_tests_positive_df.columns:
    if str(col) in confirmed_df.columns:
        similar_columns_list_confirmed.append(col)

In [34]:
# Taking only the columns in with similar columns in both dataframes
jp_tests_positive_df = jp_tests_positive_df[similar_columns_list_confirmed]

similar_columns_list_confirmed = [str(i) for i in similar_columns_list_confirmed]
indexes = ['Country/Region','Country Code','Lat','Long']
lst = list(similar_columns_list_confirmed)
for i in lst:
    indexes.append(i)

final_confirmed_df = confirmed_df[indexes]

In [36]:
# Summing up the total positive cases across all the days
total_tests_positive_df = jp_tests_positive_df.sum(axis=1)
total_tests_positive_df = total_tests_positive_df.to_frame()
total_tests_positive_df.columns = ['Total Positives']

#was facing some problems in directly merging the dataframes as is due to column names not displaying as desired, so writing into a csv
total_tests_positive_df.to_csv("datasets/positives.csv")

#Summing up the total confirmed cases across all days 
final_confirmed_df['Total Confirmed'] = final_confirmed_df[lst].sum(axis=1)
total_confirmed_df = final_confirmed_df[['Country/Region','Country Code','Lat','Long','Total Confirmed']]

total_confirmed_df.head()

,Country/Region,Country Code,Lat,Long,Total Confirmed
0,Afghanistan,AFG,33.00,65.00,15197
1,Albania,ALB,41.15,20.17,12060
2,Algeria,DZA,28.03,1.66,46533
3,Andorra,AND,42.51,1.52,15989
4,Angola,AGO,-11.20,17.87,440


In [37]:
# Reading the earlier total positives dataset and renaming the columns according to merge into a single dataset having the total confirmed cases and total positives
#based on countries
final_positives_df = pd.read_csv('datasets/positives.csv')
final_positives_df.columns = ['Country/Region','Total Positives']

final_positives_df.head()

,Country/Region,Total Positives
0,Afghanistan,13417.0
1,Africa,342286.0
2,Albania,11433.0
3,Algeria,43575.0
4,Andorra,15314.0


In [39]:
#Final data set having aggregated confirmed and positive cases
final_df = pd.merge(total_confirmed_df,final_positives_df,on='Country/Region')
final_df.to_csv("datasets/aggregated_counts.csv")

final_df.head()

,Country/Region,Country Code,Lat,Long,Total Confirmed,Total Positives
0,Afghanistan,AFG,33.00,65.00,15197,13417.0
1,Albania,ALB,41.15,20.17,12060,11433.0
2,Algeria,DZA,28.03,1.66,46533,43575.0
3,Andorra,AND,42.51,1.52,15989,15314.0
4,Angola,AGO,-11.20,17.87,440,409.0
